<center>Elective 4</center>
<center>Deep Learning</center>

## **Advisor:** Jhun Brian Andam
## **Student:** Christian Jay Baguio

<div class="alert alert-block alert-success" style="font-family: Times New Roman">
    <h4><strong>Laboratory Task 4</strong></h4>

<p style="font-family:Times New Roman; text-align:justify; font-size:15px">
    <b>Instruction:</b> Train a linear regression model in PyTorch using a regression dataset. Use the following parameters.
</p>

<ul>
    <li>Criterion: MSE Loss</li>
    <li>Fully Connected Layers x 2</li>
    <li>Batch Size: 8</li>
    <li>Optimizer: SGD</li>
    <li>Epoch: 1000</li>
</ul>
</div>

# Imports

In [1]:
import pandas as pd
import torch
import numpy as np
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn as nn
import random
import os
import sys

sheets = pd.ExcelFile('UPCAT_DATA2.xlsx')
sheets.sheet_names

['AllData', '100']

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
#We can check whether we have gpu
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
print("Device: ", device)

Device:  cuda:0


In [4]:
df = pd.read_excel('UPCAT_DATA2.xlsx', sheet_name=1)
df

,Eng7,Eng8,Eng9,Math7,Math8,Math9,Sci7,Sci8,Sci9,GWA7,GWA8,GWA9,IQ,SA,Target
0,77,82,86,94,87,92,84,80,92,1.4764,1.5045,1.4098,55,28,1
1,57,57,59,55,46,57,53,45,53,2.6557,2.7000,2.7008,38,19,0
2,55,52,55,57,53,51,57,62,51,2.6132,2.6818,2.7172,17,9,0
3,69,78,78,87,79,79,83,82,79,1.7217,1.6136,1.7049,45,28,0
4,64,71,68,71,57,54,60,59,55,2.4245,2.4318,2.4713,67,56,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,66,77,77,78,75,70,87,83,77,1.7594,1.8364,1.9221,43,39,1
96,76,74,77,76,65,65,79,57,64,1.8726,2.1455,2.2500,84,72,1
97,67,77,74,75,67,67,76,72,67,2.0566,1.9727,2.0492,69,56,1
98,59,56,63,63,51,60,74,55,61,2.2736,2.4273,2.2951,50,22,0


In [119]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Eng7    100 non-null    int64  
 1   Eng8    100 non-null    int64  
 2   Eng9    100 non-null    int64  
 3   Math7   100 non-null    int64  
 4   Math8   100 non-null    int64  
 5   Math9   100 non-null    int64  
 6   Sci7    100 non-null    int64  
 7   Sci8    100 non-null    int64  
 8   Sci9    100 non-null    int64  
 9   GWA7    100 non-null    float64
 10  GWA8    100 non-null    float64
 11  GWA9    100 non-null    float64
 12  IQ      100 non-null    int64  
 13  SA      100 non-null    int64  
 14  Target  100 non-null    int64  
dtypes: float64(3), int64(12)
memory usage: 11.8 KB


In [5]:
x_prepro = df[['GWA7', 'GWA8', 'GWA9']].astype('float32')
y_prepro = df[['IQ', 'SA']].astype('float32')

In [6]:
def seed_everything(seed=42):
  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

In [7]:
x_train = x_prepro.to_numpy()
y_train = y_prepro.to_numpy()

In [8]:
inputs = torch.from_numpy(x_train)
targets = torch.from_numpy(y_train)
print(inputs.size())
print(targets.size())

torch.Size([100, 3])
torch.Size([100, 2])


In [9]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[1.4764, 1.5045, 1.4098],
         [2.6557, 2.7000, 2.7008],
         [2.6132, 2.6818, 2.7172]]),
 tensor([[55., 28.],
         [38., 19.],
         [17.,  9.]]))

In [10]:
# Define data loader
batch_size = 8
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [11]:
x, y = next(iter(train_dl))

In [12]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[2.1698, 2.2045, 2.2623],
        [1.5000, 1.6500, 1.6598],
        [1.3821, 1.5182, 1.5246],
        [2.5896, 2.6045, 2.5943],
        [2.2547, 2.3545, 2.3689],
        [2.5330, 2.6727, 2.5738],
        [1.7594, 1.8364, 1.9221],
        [2.5377, 2.7364, 2.7500]])
tensor([[45., 14.],
        [71., 68.],
        [84., 76.],
        [67., 52.],
        [33., 29.],
        [52., 27.],
        [43., 39.],
        [25., 22.]])


In [13]:
# Define model

seed_everything()
model = nn.Linear(3, 1)  #nn.Linear assume this shape (in_features, out_features)
print(model.weight)
print(model.weight.size()) # (out_features, in_features)
print(model.bias)
print(model.bias.size()) #(out_features)

Parameter containing:
tensor([[ 0.4414,  0.4792, -0.1353]], requires_grad=True)
torch.Size([1, 3])
Parameter containing:
tensor([0.5304], requires_grad=True)
torch.Size([1])


In [14]:
# Parameters
list(model.parameters())  #model.param returns a generator

[Parameter containing:
 tensor([[ 0.4414,  0.4792, -0.1353]], requires_grad=True),
 Parameter containing:
 tensor([0.5304], requires_grad=True)]

In [15]:
#we can print the complexity by the number of parameters
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

4


In [131]:
# Generate predictions
preds = model(inputs)
preds

tensor([[1.7123],
        [2.6312],
        [2.6015],
        [1.8330],
        [2.4316],
        [2.5230],
        [1.8447],
        [1.6827],
        [2.1586],
        [2.5968],
        [1.8779],
        [2.3204],
        [1.7318],
        [2.5811],
        [2.2614],
        [2.0187],
        [2.0808],
        [2.7239],
        [2.0447],
        [1.9764],
        [1.6611],
        [2.3138],
        [2.4445],
        [1.7480],
        [1.9902],
        [1.7873],
        [1.9236],
        [1.6707],
        [1.8532],
        [1.9008],
        [2.4560],
        [2.1029],
        [2.3322],
        [2.4698],
        [2.2087],
        [2.2472],
        [2.1191],
        [2.3009],
        [1.9223],
        [2.4779],
        [1.8469],
        [1.6384],
        [1.5899],
        [1.7587],
        [1.8635],
        [2.3079],
        [1.8617],
        [2.2385],
        [2.1907],
        [1.6416],
        [1.9179],
        [1.8351],
        [2.5337],
        [1.9544],
        [2.5357],
        [1

In [132]:
criterion_mse = nn.MSELoss()
criterion_softmax_cross_entropy_loss = nn.CrossEntropyLoss()

In [133]:
mse = criterion_mse(preds, targets)
print(mse)
print(mse.item())  ##print out the loss number

tensor(3545.1160, grad_fn=<MseLossBackward0>)
3545.115966796875


C:\Users\ATARA\anaconda3\envs\tor_env\lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([100, 2])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [134]:
# Define optimizer
#momentum update the weight based on past gradients also, which will be useful for getting out of local max/min
#If our momentum parameter was $0.9$, we would get our current grad + the multiplication of the gradient 
#from one time step ago by $0.9$, the one from two time steps ago by $0.9^2 = 0.81$, etc.

opt = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

In [135]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            xb.to(device) #move them to gpu if possible, if not, it will be cpu
            yb.to(device)
                    
            # 1. Predict
            pred = model(xb)
                      
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Calculate gradient
            opt.zero_grad()  #if not, the gradients will accumulate
            loss.backward()
            
            # Print out the gradients.
            # print ('dL/dw: ', model.weight.grad) 
            # print ('dL/db: ', model.bias.grad)
            
            # 4. Update parameters using gradients
            opt.step()
            
        # Print the progress
        if (epoch+1) % 10 == 0:
            sys.stdout.write("\rEpoch [{}/{}], Loss: {:.4f}".format(epoch+1, num_epochs, loss.item()))

In [136]:
#train for 1000 epochs
fit(1000, model, criterion_mse, opt, train_dl)

C:\Users\ATARA\anaconda3\envs\tor_env\lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([8, 2])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\ATARA\anaconda3\envs\tor_env\lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([4, 2])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1000/1000], Loss: 628.7933

In [137]:
#experiment train
fit(5000, model, criterion_mse, opt, train_dl)

Epoch [5000/5000], Loss: 149.8172

In [138]:
# Generate predictions
preds = model(inputs)
loss = criterion_mse(preds, targets)
print(loss.item())

422.1947326660156


In [139]:
preds

tensor([[70.9591],
        [34.0665],
        [35.4344],
        [64.8952],
        [41.7719],
        [39.2117],
        [64.4471],
        [71.7784],
        [53.9218],
        [35.6109],
        [64.1421],
        [46.6637],
        [70.4305],
        [36.9091],
        [49.7764],
        [58.9846],
        [55.0389],
        [28.1597],
        [58.3365],
        [60.5468],
        [73.3634],
        [46.6506],
        [42.9137],
        [69.0885],
        [60.6544],
        [68.3813],
        [63.2960],
        [71.7375],
        [65.2255],
        [63.5105],
        [42.4303],
        [56.8123],
        [48.8467],
        [40.9365],
        [51.1067],
        [51.0582],
        [54.8983],
        [48.7148],
        [61.6714],
        [40.6871],
        [65.5682],
        [74.1401],
        [76.0598],
        [70.0077],
        [63.9238],
        [45.7314],
        [63.8281],
        [49.7475],
        [52.2433],
        [74.1140],
        [63.1223],
        [63.9306],
        [37.

In [140]:
targets

tensor([[55., 28.],
        [38., 19.],
        [17.,  9.],
        [45., 28.],
        [67., 56.],
        [79., 13.],
        [89., 67.],
        [94., 55.],
        [90., 83.],
        [25.,  9.],
        [81., 76.],
        [29.,  8.],
        [91., 78.],
        [52., 27.],
        [57., 23.],
        [87., 76.],
        [62., 23.],
        [57., 23.],
        [60., 27.],
        [62., 47.],
        [89., 71.],
        [45., 23.],
        [55., 66.],
        [62., 69.],
        [67., 71.],
        [87., 58.],
        [38., 34.],
        [65., 23.],
        [87., 36.],
        [81., 55.],
        [81., 74.],
        [73., 40.],
        [73., 32.],
        [33., 36.],
        [67., 44.],
        [10.,  6.],
        [62., 26.],
        [73., 72.],
        [67., 26.],
        [57., 26.],
        [67., 76.],
        [84., 71.],
        [90., 88.],
        [71., 68.],
        [75., 60.],
        [48., 23.],
        [88., 79.],
        [45., 14.],
        [50.,  8.],
        [88., 90.],
